# Remote Asset Management Indicator (RAMI)

This Jupyter notebook summarises the process for testing the RAMI device.

The RAMI device uses the following libraries (The arduino-LoRa and SparkFun_ADXL345_Arduino libraries have been forked and modified):

| Library                                                      | Version                                                            |
|--------------------------------------------------------------|--------------------------------------------------------------------|
| https://github.com/sandeepmistry/arduino-LoRa                | https://github.com/uwasystemhealth/arduino-LoRa/releases/tag/0.8.0 |
| https://github.com/Makuna/Rtc                                | https://github.com/Makuna/Rtc/releases/tag/2.3.3                   |
| https://github.com/sparkfun/SparkFun_ADXL345_Arduino_Library | https://github.com/uwasystemhealth/SparkFun_ADXL345_Arduino_Library/releases/tag/1.1.0 |



## Hardware

The circuit board was designed using [Eagle CAD](https://www.autodesk.com/products/eagle/overview).

The directory [hardware/EagleCAD](hardware/EagleCAD) contains the board, schematic and libraries.

PDF versions of the schematic and the board can be found here:

* [PCB](hardware/pdfs/PCB.pdf)

* [Schematic](hardware/pdfs/schematic.pdf)

A primary objective for the design was to make the board as small as possible in order to keep manufacturing costs low and to increase the versatility of the device. Some concessions were made in regards to the size of the board to allow for breakout headers for the unused pins on the board. This was to allow the use of the board as a testing platform for future designs incorporating other sensors or different functionality.

### Hardware Interconnects

The ADXL345 and RFM95W were linked to the ATMega328P using 3-wire SPI  in order to minimise bus power consumption<sup>1</sup>. 
The DS3231M was connected over I2C with external pullup resistors as it is the only connection offered by the [DS3231M module](https://www.maximintegrated.com/en/products/analog/real-time-clocks/DS3231M.html). 
The ADXL345 interrupt pin is connected to pin 32 (Arduino Pin 2) of the ATMega328P and internally pulled high. 
The RTC interrupt pin is connected to pin 1 (Arduino Pin 3) of ATMega328P and internally pulled high. 
Digital output pins from the ATMega328P were utilised for a variety of functions defined in the table below.

[1]: T. Solheim and M. Grannæs, "A comparison of serial interfaces on energy critical systems," 2015 Nordic Circuits and Systems Conference (NORCAS): NORCHIP \& International Symposium on System-on-Chip (SoC), Oslo, 2015, pp. 1-4.

### Pin Mappings for ATMega328P


| Arduino Pin | Mapped Function               |
|-------------|-------------------------------|
| 5           | RTC reset pin                 |
| 6           | LoRa interrupt pin            |
| 7           | Accelerometer chip select pin |
| 9           | LoRa chip Select Pin          |
| 10          | LoRa Reset Pin                |

### Board Arrangement

![Layout of the RAMI board, screenshot from Autodesk EAGLE.](figures/board_layout.png "Layout of the RAMI board")


Routing of power traces has been kept to the outside of the board as much as possible as the power traces predominantly run vertically up the plane. 
Data traces are predominantly in the center of the board, vertical traces are run  along the top layer of the board and horizontal traces along the bottom layer in order to minimise the number of vias required. 
20mil traces have been used in all instances, although this is much wider than the required trace width (5mil) it provides a 4-fold reduction in trace resistance. 
The ATMega328 was placed centrally on the board to minimise the length of data traces, with the longest data trace being 47mm for the ISP clock signal. 
The crystal oscillator has also been placed in close proximity to the ATMega328P as it is the highest frequency signal run along a trace on the board. 


The size of the board could be further reduced by placing the LoRa module on the underside of the board, however for ease of testing, development and manufacture it was decided to keep all components on one plane in this iteration. 
The vertical header along the top right side of the board provides a breakout for the unused pins on the ATMega328P and the second accelerometer interrupt. 
Most of these pins are multipurpose digital/analog pins, this allows for the connection of both analog and digital sensors to the platform. 
The RFM95W digital outputs are broken out on the bottom right of the board. 
The behaviour of these pins is configurable through SPI and allows for signals such as; end transmission, packet received, and channel busy to be used as inputs to other devices ([RFM95/96/97/98(W) - Low Power Long Range Transceiver Module'' HopeRF, 2006](https://cdn.sparkfun.com/assets/learn_tutorials/8/0/4/RFM95_96_97_98W.pdf)). 
The horizontal header along the top right of the board is a serial communications header. 
The layout order for the serial communications header is: DTR, MRX, MTX, 3.3V, CTR, GND. 
This layout makes it compatible with the headers on off the shelf 3.3V serial adapters without the need for manual wiring. 
The header on the top left is an In-System Programming (ISP) header, this allows for the ATMega328P to be programmed directly without the need for a boot loader. 
Using this method saves on program space and decreases the startup time for the device. 
The LoRa module in the center of the board towards the lower third has been positioned upside down relative to the board, the main reason for this decision is to keep the antenna trace away from all clocked data signals. 
The antenna trace is the single header towards the bottom right of the board. 
This trace was kept as short as possible and is ground shielded on its plane to reduce interference. 
The power control circuit is located on the bottom left of the board. 

The easiest method of manufacture of this board is via reflow soldering for all SMD components, followed by iron soldering of the through hole components and LoRa module. 
A working board was also developed using a fixed heat gun to mount the ADXL345 and using a soldering iron to mount all other components.

### Bill of Materials

| Designator | Description           | Part No.           |  Value | Package    | Quantity |  Price |
|------------|-----------------------|--------------------|--------|------------|----------|--------|
| R1, R4, R5 | SMD Resistors         | CRGP0603F10K       |   10k  | 0603       |        3 |  0.153 |
| R2, R3     | SMD Resistors         | CRCW06034K70JNEAC  |  4.7k  | 0603       |        2 |  0.153 |
| C1, C2     | SMD Capacitors        | VJ0603A180JXACW1BC |   18pF | 0603       |        2 |  0.153 |
| C3, C4     | SMD Capacitors        | 885012206071       |  0.1uF | 0603       |        2 |  0.153 |
| C5         | TH Capacitor          | SA305E105MAC       |    1uF | 3mm Aixial |        1 |  0.887 |
| Y1         | 8MHz Crystal          | LFXTAL069874       |    8MHz| HC-49-4H   |        1 |  0.734 |
| IC1        | 3.3V LDO Regulator    | TC1262-3.3VDB      |  3.3V  | SOT-223-3  |        1 |  0.78  |
| IC2        | ADXL345 Accelerometer | ADXL345BCCZ        |   -    | LGA (14)   |        1 | 10.61  |
| U1         | 328P Microcontroller  | ATMEGA328P-AU      |   -    | TQFP-32    |        1 |  3.07  |
| U2         | LoRa Radio Transceiver| RFM95W-915S2       | 915MHz | Castellated|        1 | 20.89  |
| RTC1       | RTC Module            | DS3231MZ+          |   -    | SOIC-8     |        1 | 11.47  |
| PCB        | Custom PCB            | -                  |   -    |     -      |     -    |  1.00  |
| Total      | -                     | -                  |   -    |     -      |     -    | 50.665 |


Pricing from au.mouser.com and Digikey, all pricing in AUD EX. GST

## Software

Here we outline how to get the arduino IDE setup to interact with the RAMI device.

RAMI:
* Install the Rtc by Makuna version 2.3.3 library
* Copy the modified libraries under lib to your libraries folder (usually ~/Documents/Arduino\/libraries on Windows)
* Select "Arduino Pro or Pro Mini" as the board type.
* Select "ATmega328P 3.3V 8MHz" as the processor.
* On any 3.3v serial adapter connect (adapter->board) TX->MTX, RX->MRX, DTR->RES, GND->GND
* Connect power to the RAMI device
* Select "Upload" to program the device

TTGO ESP32 LORA:
* Install ESP32 Board Definitions in Board Manager.
* Select board type "TTGO LoRa32-OLEDV1"
* Select "Upload" to program the device


## Communicating using TTGO LoRa device

Below is the code which can be run on the server computer with a TTGO LoRa device connected.
This code is used to test the RAMI device.

TODO :

* Explain how to program the TTGO device


In [ ]:
port = '/dev/ttyUSB0'
board = '???'

In [ ]:
sketch = 'TTGO/LoRaReceiver/LoRaReceiver.ino'

In [ ]:
print(f"arduino --board {board} --port {port} --upload {sketch}")
!arduino --board $board --port $port --upload $sketch

In [ ]:
import serial
import sys
import time
from datetime import datetime

def process_output(output, output_file=None): 
    '''
process_output takes as input a binary string from a TTGO LoRa device, 
which in turn is listening for a RAMI device.
It then interprets this binary string.
'''

    recv_packet_ids = {
        0x00: "DATA",
        0x01: "HEARTBEAT",
        0x02: "INACTIVE",
        0xff: "SERIAL_KEEPALIVE",  # Only used for debugging serial connection
    }

    mode = int(output[0])
    ID = format(output[1], '02x')
    nonce = format(int.from_bytes(output[2:4], 'little'), '02x')
    try: 
        recv_packet_ids[mode]
    except KeyError:
        print("KeyError, this can be caused by boot messages or a malformed packet")
        return
    output_str = (f"{datetime.now()}\n"
                  f"Received : {output} with length {len(output)}\n"
                  f"Received data with identifier {recv_packet_ids[mode]} "
                  f"with ID {ID} "
                  f"and nonce {nonce}\n"
         )
    if mode == 0:
        magnitude = int.from_bytes(output[5:-2], "little")
        output_str += f"Magnitude is {magnitude}\n"

    print(output_str)
        
    if not output_file:
        output_file = 'RAMI_data.txt'
    with open(output_file, 'a+') as f:
        f.write(output_str);

def lora_listen(port):
    print("Opening serial port on " + port)
    serialPort = serial.Serial(port=port, baudrate = 115200)
    print(datetime.now())
    print("Serial port opened, starting server loop")

    try:
        while(True):
            output = serialPort.read_until(b'\r\n')
            process_output(output)
            time.sleep(1)
    except KeyboardInterrupt:
        print("Stopping listening")
        serialPort.close()
        pass

In [ ]:
port = '/dev/ttyUSB0'

In [ ]:
import threading
thread = threading.Thread(target=lora_listen, args=(port,))
thread.start()
#lora_listen((port))

In [ ]:
thread.stop()

## Measuring the current draw of the RAMI

We use a [Siglent SDM3045X](https://siglentna.com/digital-multimeters/sdm3045x-digital-multimeter/) Digital Multimeter to measure the current draw of the RAMI

The [User manual](https://siglentna.com/download/2592/) for the SDM3045X does not have the SCPI commands but [this manual](https://www.batronix.com/files/Siglent/Multimeter/SDM3045X/SDM3045X_ProgrammingManual_EN.pdf) does. We can use Python tho record the current draw.



In [ ]:
import vxi11
Siglent = vxi11.Instrument('192.168.0.51')
print(Siglent.ask("*IDN?"))

In [ ]:
!which arduino

In [ ]:
!arduino --version

In [ ]:
port = '/dev/ttyUSB1'
board = 'arduino:avr:pro:cpu=8MHzatmega328'

### Transmit mode

Here we load the transmit_power program and measure the voltage and current.

In [ ]:
sketch = 'Sensor/Power_Testing/power_transmit/power_transmit.ino'

In [ ]:
print(f"arduino --board {board} --port {port} --upload {sketch}")
!arduino --board $board --port $port --upload $sketch

In [ ]:
current = float(Siglent.ask("MEAS:CURRent:DC?"))
print(current)

### Sleep mode

In [ ]:
sketch = 'Sensor/Power_Testing/power_down/power_down.ino'

In [ ]:
print(f"arduino --board {board} --port {port} --upload {sketch}")
!arduino --board $board --port $port --upload $sketch

### RAMI code

In [ ]:
sketch = 'Sensor/RAMI/RAMI.ino'

In [ ]:
print(f"arduino --board {board} --port {port} --upload {sketch}")
!arduino --board $board --port $port --upload $sketch

In [ ]:
current = Siglent.ask("READ?")
print(current)
#Siglent.ask("TRIGger:DELay:AUTO 1")

In [ ]:
Siglent.ask("INITiate")

In [ ]:
Siglent.ask("TRIGger:DELay:AUTO 1")

In [ ]:
Siglent.